In [ ]:
# metrics
# [clean accuracy, robust accuracy, top-k mask overlap under pertubed]

# vit model
# deit, vit

# dataset
# cifar10, lsun_bedroom, lsun_cat, lsun_church

# threat method

In [ ]:
# clean case

# get model
# get dataset
# loop over dataset
# get metrics: clean accuracy


In [ ]:
# poisoning case without defense

# get model
# get dataset  
# loop over dataset
# for data inside the poison set
# get the original attention mask
# poison the data with Adversarial Attack
# get the attention mask after poisoning
# get the metrics: robust accuracy, top-k mask overlap under pertubed


In [ ]:
# poisoning case with defense

# get model
# get dataset  
# loop over dataset
# for data inside the poison set
# get the original attention mask
# poison the data with Adversarial Attack with epsilon
# 1. denoise poisoned image with pretrained diffusion model with epsilon
# 2. conduct random smoothing on the denoised image with delta of R computed using (2*delta ?)
# get the attention mask after poisoning
# get the metrics: robust accuracy, top-k mask overlap under pertubed
